## 方法
- agg: 采集子图，对子图的边做总结
- passing: 对边的描述和边的两个节点信息再做一次分析，更新两个节点信息
- link prediction: 对要预测的边做预测

In [1]:
import pandas as pd
from ast import literal_eval

# 导入
sample = pd.read_csv('sample/实验样本.csv')
sample['subgraph_1hop_title'] = sample['subgraph_1hop_title'].apply(lambda x: literal_eval(x) if pd.notnull(x) else x)
sample

,pid1,pid2,title1,title2,abs1,abs2,label1,label2,subgraph_1hop,subgraph_2hop,subgraph_1hop_title,subgraph_2hop_title,link,baseline,LP,P1,P2,P3,vote
0,35.0,1033.0,Title: 4 Implementing Application Specific Rou...,Title: The Evolutionary Cost of Learning,"Abstract: To implement a specific application,...",Abstract: Traits that are acquired by members ...,Genetic_Algorithms,Genetic_Algorithms,"[[82920, 210871, 210872, 210872, 35, 45605, 41...","[[82920, 210871, 82920, 210872, 210872, 210872...","[[Title: A Survey of Evolution Strategies, cat...","[['Title: A Survey of Evolution Strategies, ca...",1.0,1.0,0.00,1.00,1.00,1.0,1.0
1,35.0,662416.0,Title: 4 Implementing Application Specific Rou...,Title: USING SMOOTHING SPLINE ANOVA TO EXAMINE...,"Abstract: To implement a specific application,...",Abstract: This paper presents recent developme...,Genetic_Algorithms,Probabilistic_Methods,"[[6910, 82920, 263553, 210871, 210872, 210872,...","[[6910, 82920, 263553, 210871, 82920, 210872, ...","[[Title: A Survey of Evolution Strategies, cat...","[['Title: A Survey of Evolution Strategies, ca...",0.0,0.0,0.00,0.00,0.00,0.0,0.0
2,35.0,103515.0,Title: 4 Implementing Application Specific Rou...,Title: The Royal Road for Genetic Algorithms: ...,"Abstract: To implement a specific application,...",Abstract: Genetic algorithms (GAs) play a majo...,Genetic_Algorithms,Genetic_Algorithms,"[[82920, 210871, 210872, 210872, 8865, 35, 35]...","[[82920, 210871, 82920, 210872, 210872, 210872...","[[Title: A Survey of Evolution Strategies, cat...","[['Title: A Survey of Evolution Strategies, ca...",1.0,1.0,1.00,1.00,1.00,1.0,1.0
3,35.0,436796.0,Title: 4 Implementing Application Specific Rou...,Title: Some Extensions of the K-Means Algorith...,"Abstract: To implement a specific application,...",Abstract: In this paper we present some extens...,Genetic_Algorithms,Neural_Networks,"[[82920, 210871, 910, 210872, 210872, 35, 1365...","[[82920, 210871, 82920, 910, 210872, 1365, 210...","[[Title: A Survey of Evolution Strategies, cat...","[['Title: A Survey of Evolution Strategies, ca...",0.0,0.0,0.00,0.00,0.00,0.0,0.0
4,35.0,1050679.0,Title: 4 Implementing Application Specific Rou...,Title: Diplomarbeit A Genetic Algorithm for th...,"Abstract: To implement a specific application,...",Abstract: We describe an integrated problem so...,Genetic_Algorithms,Genetic_Algorithms,"[[82920, 210871, 210872, 210872, 427606, 35, 3...","[[82920, 30817, 210871, 82920, 29492, 4330, 21...","[[Title: A Survey of Evolution Strategies, cat...","[['Title: A Survey of Evolution Strategies, ca...",1.0,0.0,1.00,0.00,0.00,0.0,0.0
5,35.0,139547.0,Title: 4 Implementing Application Specific Rou...,Title: On Centering Neural Network Weight Updates,"Abstract: To implement a specific application,...",Abstract: Technical Report IDSIA-19-97 Abstrac...,Genetic_Algorithms,Neural_Networks,"[[82920, 210871, 210872, 210872, 626999, 5348,...","[[82920, 8581, 210871, 82920, 210872, 210872, ...","[[Title: A Survey of Evolution Strategies, cat...","[['Title: A Survey of Evolution Strategies, ca...",0.0,0.0,0.00,1.00,1.00,1.0,1.0
6,35.0,1103960.0,Title: 4 Implementing Application Specific Rou...,Title: Surgery,"Abstract: To implement a specific application,...",Abstract: Object localization has applications...,Genetic_Algorithms,Genetic_Algorithms,"[[82920, 210871, 210872, 210872, 3229, 33895, ...","[[82920, 30817, 35, 210871, 14062, 82920, 2949...","[[Title: A Survey of Evolution Strategies, cat...",[['Title: Generalized Markov Decision Processe...,1.0,1.0,0.00,1.00,1.00,1.0,1.0
7,35.0,227178.0,Title: 4 Implementing Application Specific Rou...,Title: Evolving Visual Routines,"Abstract: To implement a specific application,...",Abstract: Traditional machine vision assumes t...,Genetic_Algorithms,Genetic_Algorithms,"[[82920, 210871, 210872, 210872, 44455, 35], [...","[[82920, 210871, 82920, 210872, 210872, 210872...",[[Title: Coevolving High-Level Representations...,[['Title: Coevolving High-Level Representati

### summary_prompt

In [2]:
# 对子图做总结，放入description里

# A and B have a cited relationship

def verbal_subgraph(subgraph):
    result = ""
    for i in range(len(subgraph[0])):
        title1, title2 = subgraph[0][i], subgraph[1][i]
        verbal_cite = f'{i+1}."{title1}" cites "{title2}" '
        result += "\n" + verbal_cite + "\n"
    return result


i = 0
print(i)
subgraph = sample.loc[i, 'subgraph_1hop_title']
subgraph_prompt = f"""
Given citation relationships, shown as follows:
{verbal_subgraph(subgraph)}
Please summarize the information of these citation relationships in no more than three sentences.
Do not generate ordinal words like first, second, third...
Answer:
"""

print(subgraph_prompt)
print('==========================================================================================================')

0

Given citation relationships, shown as follows:

1."Title: A Survey of Evolution Strategies, category:Genetic_Algorithms" cites "Title: 4 Implementing Application Specific Routines  Genetic algorithms in search, optimization, and machine learning. Reading, MA: Addison-Wesley., category:Genetic_Algorithms" 

2."Title: Coevolving High-Level Representations, category:Genetic_Algorithms" cites "Title: 4 Implementing Application Specific Routines  Genetic algorithms in search, optimization, and machine learning. Reading, MA: Addison-Wesley., category:Genetic_Algorithms" 

3."Title: An Evolutionary Algorithm that Constructs Recurrent Neural Networks, category:Genetic_Algorithms" cites "Title: 4 Implementing Application Specific Routines  Genetic algorithms in search, optimization, and machine learning. Reading, MA: Addison-Wesley., category:Genetic_Algorithms" 

4."Title: An Evolutionary Algorithm that Constructs Recurrent Neural Networks, category:Genetic_Algorithms" cites "Title: Coevol

### prediction prompt

In [3]:
title1, abstract1, category1 = sample.loc[i, 'title1'], sample.loc[i, 'abs1'], sample.loc[i, 'label1']
title2, abstract2, category2 = sample.loc[i, 'title2'], sample.loc[i, 'abs2'], sample.loc[i, 'label2']

description = """

The citations form a network where "A Survey of Evolution Strategies" cites "Implementing Application Specific Routines," which in turn cites "Issues in Evolutionary Robotics" and "Coevolving High-Level Representations." "An Evolutionary Algorithm that Constructs Recurrent Neural Networks" cites both "Implementing Application Specific Routines" and "Coevolving High-Level Representations," highlighting interconnections among these works in the context of genetic algorithms and evolutionary algorithms applied to machine learning. Additionally, "Learning and evolution in neural networks" cites "The Evolutionary Cost of Learning," emphasizing the integration of evolutionary principles into neural network learning processes.
"""

prediction_prompt = f"""
Given two papers and their their citation information shown as follows:

paper1:
title:{title1}
abstract:{abstract1}
category:{category1}

paper2:
title:{title2}
abstract:{abstract2}
category:{category2}

citation information:{description}

please answer the following question:
Is there an existing citation relationship between "{title1}, category: {category1}" and "{title2}, category: {category2}"
Do not give any reasoning or logic for your answer. Answer Yes if they have cited relationship, otherwise, answer No.
answer:
"""

print(prediction_prompt)


Given two papers and their their citation information shown as follows:

paper1:
title:Title: 4 Implementing Application Specific Routines  Genetic algorithms in search, optimization, and machine learning. Reading, MA: Addison-Wesley.
abstract:Abstract: To implement a specific application, you should only have to change the file app.c. Section 2 describes the routines in app.c in detail. If you use additional variables for your specific problem, the easiest method of making them available to other program units is to declare them in sga.h and external.h. However, take care that you do not redeclare existing variables. Two example applications files are included in the SGA-C distribution. The file app1.c performs the simple example problem included with the Pascal version; finding the maximum of x 10 , where x is an integer interpretation of a chromosome. A slightly more complex application is include in app2.c. This application illustrates two features that have been added to SGA-C. T

### 补充

- 直接把子图信息扔到citation information里

In [90]:
i = 19
subgraph = sample.loc[i, 'subgraph_1hop_title']
subgraph_prompt = f"""
{verbal_subgraph(subgraph)}
"""

title1, abstract1, category1 = sample.loc[i, 'title1'], sample.loc[i, 'abs1'], sample.loc[i, 'label1']
title2, abstract2, category2 = sample.loc[i, 'title2'], sample.loc[i, 'abs2'], sample.loc[i, 'label2']

prediction_prompt = f"""
Given two papers and their their citation information shown as follows:

paper1:
title:{title1}
abstract:{abstract1}
category:{category1}

paper2:
title:{title2}
abstract:{abstract2}
category:{category2}
citation information:{subgraph_prompt}
please answer the following question:
Is there an existing citation relationship between "{title1}, category: {category1}" and "{title2}, category: {category2}"
Do not give any reasoning or logic for your answer. Answer Yes if they have cited relationship, otherwise, answer No.
answer:
"""

print(prediction_prompt)


Given two papers and their their citation information shown as follows:

paper1:
title:Title: 4 Implementing Application Specific Routines  Genetic algorithms in search, optimization, and machine learning. Reading, MA: Addison-Wesley.
abstract:Abstract: To implement a specific application, you should only have to change the file app.c. Section 2 describes the routines in app.c in detail. If you use additional variables for your specific problem, the easiest method of making them available to other program units is to declare them in sga.h and external.h. However, take care that you do not redeclare existing variables. Two example applications files are included in the SGA-C distribution. The file app1.c performs the simple example problem included with the Pascal version; finding the maximum of x 10 , where x is an integer interpretation of a chromosome. A slightly more complex application is include in app2.c. This application illustrates two features that have been added to SGA-C. T